# MLOps HW1 — Создание воспроизводимого ML-pipeline с DVC и MLflow
Автор: Panagiotou Elina

Цель работы:
- построить воспроизводимый ML-конвейер,
- использовать DVC для управления данными и пайплайнами,
- логировать эксперименты в MLflow,
- запускать обучение одной командой `dvc repro`.


In [7]:
# устанавливаем необходимые зависимости 
!pip install dvc mlflow pandas scikit-learn pyyaml joblib

In [44]:
requirements_text = """
pandas
scikit-learn
pyyaml
joblib
dvc
mlflow
"""

with open("requirements.txt", "w", encoding="utf-8") as f:
    f.write(requirements_text.strip() + "\n")

print("requirements.txt создан.")


requirements.txt создан.


In [8]:
# создает структуру проекта - папки как в задании

import os

os.makedirs("data/raw", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)
os.makedirs("src", exist_ok=True)

print("Папки созданы.")


Папки созданы.


В рамках данного домашнего задания выбран датасет **Iris**, поскольку он является одним из наиболее классических и удобных наборов данных для демонстрации MLOps-подходов:

### 1. Лёгкость и компактность  
Набор данных состоит всего из **150 наблюдений** и **4 числовых признаков**, что делает его идеальным для воспроизводимых экспериментов. Нет долгого обучения, нет больших файлов, пайплайн работает быстро.

### 2. Отсутствие необходимости в глубокой предобработке  
Данные уже чистые:  
- нет пропусков,  
- нет категориальных признаков,  
- все признаки измерены в одном масштабе.  
Это позволяет сосредоточиться именно на инструментах MLOps (DVC, MLflow), а не на длительной подготовке данных.

### 3. Хорошо подходит для базовых моделей  
Задача — **многоклассовая классификация (3 класса ирисов)**.  
Даже простые модели (логистическая регрессия, дерево решений) дают высокую точность → удобно сравнивать эксперименты через MLflow.

### 4. Широко известный и стандартизованный датасет  
Он входит в `sklearn.datasets`, так что его можно легко загрузить программно, не скачивая файлы вручную, что упрощает интеграцию с DVC.

### 5. Идеален для демонстрации воспроизводимого ML-пайплайна  
- небольшой размер → удобно версионировать в DVC;  
- быстрые эксперименты → легко логировать в MLflow;  
- стабильный результат → удобно тестировать воспроизводимость через `dvc repro`.



## Что содержит датасет Iris

Датасет был собран британским статистиком **Рональдом Фишером** (1936) и состоит из измерений лепестков (petal) и чашелистиков (sepal) трёх видов ирисов:

**1. Iris setosa**  
**2. Iris versicolor**  
**3. Iris virginica**

Каждая строка — одно цветочное растение.  
Каждый объект характеризуется 4 признаками:

| Признак | Описание |
|--------|----------|
| sepal length (cm) | длина чашелистика |
| sepal width (cm) | ширина чашелистика |
| petal length (cm) | длина лепестка |
| petal width (cm) | ширина лепестка |

Задача — по этим измерениям предсказать вид ириса (`target` ∈ {0, 1, 2}).

In [10]:
# загружаем датасем Ирис и созраняем в data/raw/data.csv

from sklearn.datasets import load_iris
import pandas as pd

X, y = load_iris(return_X_y=True, as_frame=True)
df = X.copy()
df["target"] = y

df.to_csv("data/raw/data.csv", index=False)
df.head()


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [11]:
# инициализируем git и dvc
!git init
!dvc init


Initialized empty Git repository in C:/Users/Elina/Desktop/Master/Ð¾Ð±ÑƒÑ‡ÐµÐ½Ð¸Ðµ/Ð¢Ð Ð•Ð¢Ð˜Ð™ Ð¡Ð•ÐœÐ•Ð¡Ð¢Ð /MLOps/HW1/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [17]:
# добавляем датасет под управление DVC
!dvc add data/raw/data.csv


To track the changes with git, run:

	git add 'data\raw\.gitignore' 'data\raw\data.csv.dvc'

To enable auto staging, run:

	dvc config core.autostage true


\u280b Checking graph



In [19]:
!tree .


Folder PATH listing for volume OS
Volume serial number is 84FF-4803
C:\USERS\ELINA\DESKTOP\MASTER\????????\?????? ???????\MLOPS\HW1
+---.dvc
¦   +---cache
¦   ¦   +---files
¦   ¦       +---md5
¦   ¦           +---21
¦   +---tmp
+---.ipynb_checkpoints
+---data
¦   +---processed
¦   +---raw
+---src


In [21]:
# создаем params.yaml для зпаписи гиперпараметров
params_text = """
base:
  random_state: 42

prepare:
  test_size: 0.2

train:
  model_type: "logreg"
  C: 1.0
  max_iter: 1000
"""

with open("params.yaml", "w") as f:
    f.write(params_text)

print("params.yaml создан.")


params.yaml создан.


In [23]:
# создаем скрипт src/prepare.py для подготовки данных для обучения
prepare_code = '''
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path
import yaml

def load_params(path="params.yaml"):
    with open(path, "r") as f:
        return yaml.safe_load(f)

def main():
    params = load_params()
    test_size = params["prepare"]["test_size"]
    random_state = params["base"]["random_state"]

    df = pd.read_csv("data/raw/data.csv")

    X = df.drop(columns=["target"])
    y = df["target"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    Path("data/processed").mkdir(parents=True, exist_ok=True)

    train_df = X_train.copy()
    train_df["target"] = y_train
    test_df = X_test.copy()
    test_df["target"] = y_test

    train_df.to_csv("data/processed/train.csv", index=False)
    test_df.to_csv("data/processed/test.csv", index=False)

if __name__ == "__main__":
    main()
'''

with open("src/prepare.py", "w") as f:
    f.write(prepare_code)

print("prepare.py создан.")


prepare.py создан.


In [25]:
# создаем скрипт src/train.py (c MLflow)
train_code = '''
import pandas as pd
import yaml
from pathlib import Path

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

import mlflow
import mlflow.sklearn

def load_params(path="params.yaml"):
    with open(path, "r") as f:
        return yaml.safe_load(f)

def main():
    params = load_params()

    model_type = params["train"]["model_type"]
    C = params["train"]["C"]
    max_iter = params["train"]["max_iter"]
    random_state = params["base"]["random_state"]

    train_df = pd.read_csv("data/processed/train.csv")
    test_df = pd.read_csv("data/processed/test.csv")

    X_train = train_df.drop(columns=["target"])
    y_train = train_df["target"]
    X_test = test_df.drop(columns=["target"])
    y_test = test_df["target"]

    model = LogisticRegression(C=C, max_iter=max_iter, random_state=random_state)

    mlflow.set_experiment("iris_classification")

    with mlflow.start_run():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)

        mlflow.log_param("model_type", model_type)
        mlflow.log_param("C", C)
        mlflow.log_param("max_iter", max_iter)
        mlflow.log_param("random_state", random_state)

        mlflow.log_metric("accuracy", acc)

        joblib.dump(model, "model.pkl")
        mlflow.log_artifact("model.pkl")

        mlflow.sklearn.log_model(model, "model")

        print("Accuracy:", acc)

if __name__ == "__main__":
    main()
'''

with open("src/train.py", "w") as f:
    f.write(train_code)

print("train.py создан.")


train.py создан.


In [29]:
#создаем dvc пайплайн - prepare 
!dvc stage add -n prepare -d src/prepare.py -d params.yaml -d data/raw/data.csv -o data/processed python src/prepare.py


Added stage 'prepare' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml 'data\.gitignore'

To enable auto staging, run:

	dvc config core.autostage true


In [31]:
# добавляем train 
!dvc stage add -n train -d src/train.py -d params.yaml -d data/processed -o model.pkl python src/train.py


Added stage 'train' in 'dvc.yaml'

To track the changes with git, run:

	git add dvc.yaml .gitignore

To enable auto staging, run:

	dvc config core.autostage true


In [35]:
!type dvc.yaml


stages:
  prepare:
    cmd: python src/prepare.py
    deps:
    - data/raw/data.csv
    - params.yaml
    - src/prepare.py
    outs:
    - data/processed
  train:
    cmd: python src/train.py
    deps:
    - data/processed
    - params.yaml
    - src/train.py
    outs:
    - model.pkl


In [37]:
!dvc repro


'data\raw\data.csv.dvc' didn't change, skipping
Running stage 'prepare':
> python src/prepare.py
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

Running stage 'train':
> python src/train.py
Accuracy: 0.9666666666666667
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.


C:\Users\Elina\anaconda3\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/28 20:50:40 INFO mlflow.tracking.fluent: Experiment with name 'iris_classification' does not exist. Creating a new experiment.
2025/11/28 20:50:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/28 20:50:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Интерпретация результата `dvc repro`

Команда `dvc repro` автоматически воспроизвела весь ML-пайплайн:

1. Проверила, изменился ли сырой датасет (`data/raw/data.csv`). Так как файл не менялся, DVC пропустил повторное добавление (`data.csv.dvc didn't change`).
2. Выполнила стадию `prepare`:
   - запустился скрипт `src/prepare.py`;
   - были сгенерированы файлы `data/processed/train.csv` и `data/processed/test.csv`.
3. Выполнила стадию `train`:
   - запустился скрипт `src/train.py`;
   - обучилась модель логистической регрессии;
   - была рассчитана метрика качества `accuracy` (в моём случае ≈ 0.97);
   - модель сохранена в файл `model.pkl` и залогирована в MLflow.
4. Был создан/обновлён файл `dvc.lock`, фиксирующий версии данных, параметров и команд для полной воспроизводимости.

Таким образом, после `git clone` → `dvc pull` → `dvc repro` любой пользователь сможет полностью воспроизвести обучение и получить те же результаты (при фиксированных версиях библиотек).


In [47]:
# создаем readme.md

readme_text = """
# MLOps HW1 — воспроизводимый ML-пайплайн с DVC и MLflow

Автор: **Panagiotou Elina**

## Цель проекта

Цель проекта — построить минимальный, но полноценный MLOps-контур для задачи классификации:

- обеспечить **воспроизводимость экспериментов**;
- настроить **версионирование данных** через DVC;
- автоматизировать процесс подготовки данных и обучения модели с помощью **DVC-пайплайна**;
- логировать **гиперпараметры, метрики и артефакты модели** в MLflow;
- запускать всё обучение одной командой `dvc repro`.

Проект использует небольшой классический датасет **Iris** (многоклассовая классификация трёх видов ирисов) и модель **логистической регрессии**.


## Описание датасета

В качестве демонстрационного датасета используется **Iris** (из библиотеки `scikit-learn`):

- 150 объектов (цветы ириса),
- 4 числовых признака:
  - sepal length (cm) — длина чашелистика,
  - sepal width (cm) — ширина чашелистика,
  - petal length (cm) — длина лепестка,
  - petal width (cm) — ширина лепестка,
- целевая переменная `target` — вид ириса: *setosa*, *versicolor*, *virginica* (3 класса).

Датасет небольшой, чистый и стандартизованный, поэтому хорошо подходит для учебной задачи по построению воспроизводимого ML-пайплайна и демонстрации работы DVC и MLflow.



## Структура проекта

\`\`\`text
.
├── data/
│   ├── raw/           # сырые данные (data.csv) — под управлением DVC
│   └── processed/     # подготовленные данные (train/test)
├── src/
│   ├── prepare.py     # подготовка данных: сплит, базовая очистка
│   └── train.py       # обучение модели и логирование в MLflow
├── dvc.yaml           # описание DVC-пайплайна (stages: prepare, train)
├── dvc.lock           # зафиксированные версии данных/артефактов для воспроизводимости
├── params.yaml        # гиперпараметры (сплит, модель и т.д.)
├── requirements.txt   # список зависимостей
└── README.md          # документация (текущий файл)
\`\`\`


## Как запустить проект

1. Клонировать репозиторий:

\`\`\`bash
git clone <URL_репозитория>
cd <имя_папки_проекта>
\`\`\`

2. Установить зависимости:

\`\`\`bash
pip install -r requirements.txt
\`\`\`

3. Подтянуть данные, версионируемые через DVC:

\`\`\`bash
dvc pull
\`\`\`

4. Воспроизвести ML-пайплайн:

\`\`\`bash
dvc repro
\`\`\`

Результат:
- сгенерируются train/test файлы в `data/processed/`,
- обучится модель (`model.pkl`),
- параметры, метрики и артефакты появятся в MLflow.


## Работа с MLflow (Tracking UI)

Запуск интерфейса:

\`\`\`bash
mlflow ui --backend-store-uri sqlite:///mlflow.db
\`\`\`

Открыть в браузере:

\`\`\`
http://127.0.0.1:5000
\`\`\`

MLflow логирует:
- параметры модели,
- гиперпараметры,
- accuracy,
- файл модели (`model.pkl`).


## DVC-пайплайн

### Stage 1: **prepare**

- читает `data/raw/data.csv`
- делит на train/test
- сохраняет в `data/processed/`

### Stage 2: **train**

- читает train/test
- обучает логистическую регрессию
- считает accuracy
- сохраняет `model.pkl`
- логирует параметры/метрики в MLflow


## Версионирование данных через DVC

- `data/raw/data.csv` добавлен через `dvc add`
- данные не хранятся в Git
- вместо этого используется:
  - `data/raw/data.csv.dvc`
  - `.gitignore`
  - локальное DVC-хранилище или удалённый remote

Команда для подтягивания данных:

\`\`\`bash
dvc pull
\`\`\`


## Итог

Проект демонстрирует полный цикл минимального MLOps-контура:

- управление данными (DVC),
- управление пайплайном (DVC),
- воспроизводимость (dvc repro + dvc.lock),
- логирование экспериментов (MLflow),
- обучение модели одной командой.

Готов для сдачи ДЗ.
"""

with open("README.md", "w", encoding="utf-8") as f:
    f.write(readme_text)

print("README.md успешно создан.")


README.md успешно создан.


## Настройка локального хранилища DVC

При первой попытке проверки воспроизводимости пайплайна возникла проблема:  
в новой (тестовой) копии проекта команда

```bash
dvc pull
````

выдавала ошибку:

```
No remote provided and no default remote set.
```

Это означало, что в исходном репозитории не было настроено локальное DVC-хранилище (remote),
поэтому DVC не знал, откуда скачивать данные (`data/raw/data.csv`, `data/processed/*`, `model.pkl`).

### Что было сделано

1. В основном проекте я создала локальное хранилище DVC:

```bash
mkdir C:\Users\Elina\DVC_storage_hw1
```

2. Настроила его как remote и сделала его default:

```bash
dvc remote add -d local_storage "C:\Users\Elina\DVC_storage_hw1"
```

3. Отправила все данные в это хранилище:

```bash
dvc push
```

4. После этого закоммитила изменения в `.dvc/config` и отправила их на GitHub:

```bash
git add .dvc/config
git commit -m "Configure DVC remote storage"
git push
```

## Проверка воспроизводимости пайплайна

Чтобы убедиться, что созданный ML-пайплайн полностью воспроизводим, я смоделировала запуск проекта в новой, «чистой» среде. Для этого были выполнены следующие шаги.

### 1. Клонирование репозитория

```bash
git clone <https://github.com/ElinaPan/MLOps-_HW1>
cd <C:\Users\Elina\Desktop\Master\обучение\ТРЕТИЙ СЕМЕСТР\MLOps\test_HW1>
````

### 2. Установка зависимостей

```bash
pip install -r requirements.txt
```

### 3. Восстановление данных через DVC

Поскольку данные находятся под управлением DVC и хранятся в заранее настроенном локальном remote-хранилище, достаточно выполнить команду:

```bash
dvc pull
```

После её выполнения в рабочей директории автоматически появились:

* `data/raw/data.csv`
* `data/processed/train.csv`
* `data/processed/test.csv`
* `model.pkl`

### 4. Воспроизведение всего пайплайна

```bash
dvc repro
```

Вывод DVC:

```
'data\raw\data.csv.dvc' didn't change, skipping
Stage 'prepare' didn't change, skipping
Stage 'train' didn't change, skipping
Data and pipelines are up to date.
```

Этот вывод означает, что:

* артефакты полностью соответствуют описанным в `dvc.lock`,
* стадии `prepare` и `train` не требуют пересборки,
* данные и код находятся в строгом соответствии с зафиксированной ранее версией пайплайна.

Это подтверждает корректность и воспроизводимость проекта.


## Итог

Проект полностью воспроизводим:
после выполнения последовательности команд
**`git clone → pip install → dvc pull → dvc repro`**
получаются те же подготовленные данные, результаты обучения, артефакты и метрики.

Это соответствует требованиям задания и демонстрирует корректную настройку DVC-пайплайна и MLflow.

